In [823]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
from sklearn.linear_model import *
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

## Load data
data_original stores original data

data2 splits Doc2Vec vector to multiple columns

data same as data2 but countries are onehot encoded

In [832]:
# load data
data_original = pd.read_csv('corepo_lookup_labelled_preprocessed_noNullLabels.csv', index_col=False)
names = data_original['name']
data_original = data_original.drop(columns=['Unnamed: 0', 'name'])
data = data_original.copy()
# data = data.drop(columns=['Unnamed: 0'])

# data = pd.get_dummies(data, columns = ['country_code'])
temp = data['description_vector'].str.replace('[', '', regex=False).str.replace(']', '', regex=False).str.split().to_list()
temp = pd.DataFrame(temp)
for i in range(0, 100):
    col_name = "dv"+str(i)
    data[col_name] = temp[i]
data = data.drop(columns=['description_vector'])
data2 = data.copy()
data2

,founding_year,funding_numeric,is_linkedin,country_code,employee_count,startup,dv0,dv1,dv2,dv3,...,dv90,dv91,dv92,dv93,dv94,dv95,dv96,dv97,dv98,dv99
0,2019.0,4700000.0,1,unknown,2,True,-1.0494092e-03,1.6099226e-03,2.5284053e-03,-6.6239148e-04,...,-1.2616039e-04,9.0520264e-04,2.4124999e-03,-3.0647549e-03,4.9191578e-03,1.4874232e-03,-4.2622220e-03,-4.9811117e-03,3.6142247e-03,-1.4059755e-03
1,1998.0,0.0,1,CAN,5,False,1.4554239e-03,2.6558102e-03,-1.1953362e-03,-1.1788297e-03,...,2.6054948e-03,4.2940318e-04,4.7795773e-05,2.0661617e-03,3.9536846e-03,-4.4104303e-03,1.4560354e-03,-5.8726221e-04,-9.9859061e-04,-1.8700099e-03
2,2015.0,182400000.0,1,CAN,3,False,1.4509535e-03,1.3987899e-03,-1.3773963e-03,1.1062956e-03,...,-6.0036749e-04,-3.5541747e-03,2.8397257e-03,7.1058872e-05,-8.9019269e-04,5.9013010e-04,-3.9178808e-03,-4.5076380e-03,1.0982275e-03,2.4652218e-03
3,1981.0,27000000.0,1,USA,5,False,3.6765509e-03,-2.2906064e-05,-4.8566400e-04,-2.9158089e-03,...,4.8886482e-03,-3.3228048e-03,-2.3203928e-03,-7.7234925e-04,-2.3988663e-04,-4.0451903e-03,-2.1371490e-03,-4.4565648e-04,1.3822239e-03,-1.4282488e-03
4,2006.0,6900000.0,1,USA,7,False,-4.6877153e-03,3.8509606e-03,-2.8387904e-05,-1.2367022e-03,...,4.8225494e-03,3.2918990e-04,4.8552631e-04,4.0964042e-03,-8.3557097e-04,-2.7602136e-03,8.6774945e-04,3.3478909e-03,2.8295422e-03,-6.3865393e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2471,2006.0,481000000.0,1,USA,4,False,-7.67612437e-05,2.56360765e-03,-3.86112044e-03,3.37789836e-03,...,1.18470727e-03,4.45345929e-03,1.76604267e-03,3.19888582e-03,1.06968284e-04,-1.39952125e-03,4.93509835e-03,-1.00720313e-03,-1.71462982e-03,-4.93891956e-03
2472,2003.0,55000000.0,1,USA,4,False,-1.1639408e-03,-1.0963881e-03,8.3642302e-04,1.3545924e-03,...,-1.3499036e-03,4.2441483e-03,-1.6905534e-03,-1.2343648e-03,-2.1577075e-03,-3.8241080e-03,-3.2053853e-03,4.1546370e-03,1.3390785e-03,-1.7110094e-03
2473,2005.0,840300000.0,1,NLD,4,False,1.0636229e-03,-3.6522693e-03,3.7946223e-04,-3.5712162e-03,...,1.2963051e-03,-4.8029441e-03,9.6474883e-05,3.8486719e-06,-4.8979610e-04,-3.2605692e-03,-3.6044091e-03,-4.6273619e-03,-4.6609933e-03,4.2935777e-03
2474,2007.0,20300000.0,1,USA,2,False,-4.8678126e-03,4.2776936e-03,1.8291145e-03,-1.3382930e-03,...,-4.8744041e-03,-4.4790488e-03,-2.6001704e-03,1.6880685e-03,-3.3709195e-03,2.7019596e-03,-1.5989543e-04,-2.3680700e-03,4.8513184e-03,-2.3024541e-03


In [3]:
data = pd.get_dummies(data, columns = ['country_code'])
data

,founding_year,funding_numeric,is_linkedin,employee_count,startup,dv0,dv1,dv2,dv3,dv4,...,country_code_SLV,country_code_SRB,country_code_SVN,country_code_SWE,country_code_THA,country_code_TUR,country_code_URY,country_code_USA,country_code_ZAF,country_code_unknown
0,2019.0,4700000.0,1,2,True,-1.0494092e-03,1.6099226e-03,2.5284053e-03,-6.6239148e-04,-5.1362516e-04,...,0,0,0,0,0,0,0,0,0,1
1,1998.0,0.0,1,5,False,1.4554239e-03,2.6558102e-03,-1.1953362e-03,-1.1788297e-03,3.4765108e-03,...,0,0,0,0,0,0,0,0,0,0
2,2015.0,182400000.0,1,3,False,1.4509535e-03,1.3987899e-03,-1.3773963e-03,1.1062956e-03,1.1308545e-03,...,0,0,0,0,0,0,0,0,0,0
3,1981.0,27000000.0,1,5,False,3.6765509e-03,-2.2906064e-05,-4.8566400e-04,-2.9158089e-03,-3.4179194e-03,...,0,0,0,0,0,0,0,1,0,0
4,2006.0,6900000.0,1,7,False,-4.6877153e-03,3.8509606e-03,-2.8387904e-05,-1.2367022e-03,9.2594209e-04,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2471,2006.0,481000000.0,1,4,False,-7.67612437e-05,2.56360765e-03,-3.86112044e-03,3.37789836e-03,1.96688529e-03,...,0,0,0,0,0,0,0,1,0,0
2472,2003.0,55000000.0,1,4,False,-1.1639408e-03,-1.0963881e-03,8.3642302e-04,1.3545924e-03,9.6043351e-04,...,0,0,0,0,0,0,0,1,0,0
2473,2005.0,840300000.0,1,4,False,1.0636229e-03,-3.6522693e-03,3.7946223e-04,-3.5712162e-03,-3.2107830e-03,...,0,0,0,0,0,0,0,0,0,0
2474,2007.0,20300000.0,1,2,False,-4.8678126e-03,4.2776936e-03,1.8291145e-03,-1.3382930e-03,-5.9233425e-04,...,0,0,0,0,0,0,0,1,0,0


## Baseline model
Classifies organizations less than 3 years old as startups

In [240]:
predicted = data['founding_year']>=2018
predicted

0        True
1       False
2       False
3       False
4       False
        ...  
2471    False
2472    False
2473    False
2474    False
2475    False
Name: founding_year, Length: 2476, dtype: bool

In [241]:
print("Confusion matrix: (True, False) \n", confusion_matrix(data['startup'], predicted, labels=[True, False]))
print("Accuracy: ", accuracy_score(data['startup'], predicted))
print("Precision: ", precision_score(data['startup'], predicted, pos_label=True))
print("Recall: ", recall_score(data['startup'], predicted, pos_label=True))
print("F1 score: ", f1_score(data['startup'], predicted, pos_label=True))

Confusion matrix: (True, False) 
 [[  36  460]
 [  11 1969]]
Accuracy:  0.8097738287560582
Precision:  0.7659574468085106
Recall:  0.07258064516129033
F1 score:  0.13259668508287292


## Logistic Regression

In [380]:
X = data_original
y = X['startup']
X = X.drop(columns=['description_vector', 'country_code', 'startup'])
X = X.replace({'founding_year': {0: 2021}}) 

f1, opt_model_name = -1, ""

solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
C = [10e-2, 10e-1, 10e0, 10e1, 10e2, 10e3]

for solver in solvers:
    for c in C:
        print("solver: ", solver, "C: ", c)
        model = make_pipeline(StandardScaler(), LogisticRegression(solver=solver, C=c))
        #             model.fit(X_train, y_train)
        #             predicted = model.predict(X_test)
        predicted = cross_val_predict(model, X, y, cv=5)
        print("Confusion matrix: (True, False) \n", confusion_matrix(y, predicted, labels=[True, False]))
        print("Accuracy: ", accuracy_score(y, predicted))
        print("Precision: ", precision_score(y, predicted, pos_label=True))
        print("Recall: ", recall_score(y, predicted, pos_label=True))
        print("F1 score: ", f1_score(y, predicted, pos_label=True))
        if f1_score(y, predicted, pos_label=True) > f1:
            f1 = f1_score(y, predicted, pos_label=True)
            opt_model_name = "solver: ", solver, "C: ", c
        print("")

solver:  newton-cg C:  0.1
Confusion matrix: (True, False) 
 [[  40  456]
 [  36 1944]]
Accuracy:  0.8012924071082391
Precision:  0.5263157894736842
Recall:  0.08064516129032258
F1 score:  0.13986013986013984

solver:  newton-cg C:  1.0
Confusion matrix: (True, False) 
 [[  67  429]
 [  52 1928]]
Accuracy:  0.805735056542811
Precision:  0.5630252100840336
Recall:  0.1350806451612903
F1 score:  0.21788617886178863

solver:  newton-cg C:  10.0
Confusion matrix: (True, False) 
 [[ 125  371]
 [  59 1921]]
Accuracy:  0.8263327948303716
Precision:  0.6793478260869565
Recall:  0.25201612903225806
F1 score:  0.36764705882352944

solver:  newton-cg C:  100.0
Confusion matrix: (True, False) 
 [[ 195  301]
 [ 100 1880]]
Accuracy:  0.8380452342487884
Precision:  0.6610169491525424
Recall:  0.39314516129032256
F1 score:  0.49304677623261695

solver:  newton-cg C:  1000.0
Confusion matrix: (True, False) 
 [[ 212  284]
 [ 118 1862]]
Accuracy:  0.8376413570274637
Precision:  0.6424242424242425
Recall:

e:\university\dissertation\code\dissertations-2021-info\kasturi_mitra_find\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
e:\university\dissertation\code\dissertations-2021-info\kasturi_mitra_find\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
e:\university\dissertation\code\dissertations-2021-info\kasturi_mitra_find\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
e:\university\dissertation\code\dissertations-2021-info\kasturi_mitra_find\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did n

Confusion matrix: (True, False) 
 [[  40  456]
 [  36 1944]]
Accuracy:  0.8012924071082391
Precision:  0.5263157894736842
Recall:  0.08064516129032258
F1 score:  0.13986013986013984

solver:  sag C:  1.0
Confusion matrix: (True, False) 
 [[  56  440]
 [  47 1933]]
Accuracy:  0.8033117932148627
Precision:  0.5436893203883495
Recall:  0.11290322580645161
F1 score:  0.18697829716193654

solver:  sag C:  10.0


e:\university\dissertation\code\dissertations-2021-info\kasturi_mitra_find\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
e:\university\dissertation\code\dissertations-2021-info\kasturi_mitra_find\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
e:\university\dissertation\code\dissertations-2021-info\kasturi_mitra_find\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
e:\university\dissertation\code\dissertations-2021-info\kasturi_mitra_find\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did n

Confusion matrix: (True, False) 
 [[  64  432]
 [  55 1925]]
Accuracy:  0.8033117932148627
Precision:  0.5378151260504201
Recall:  0.12903225806451613
F1 score:  0.208130081300813

solver:  sag C:  100.0
Confusion matrix: (True, False) 
 [[  69  427]
 [  55 1925]]
Accuracy:  0.8053311793214862
Precision:  0.5564516129032258
Recall:  0.13911290322580644
F1 score:  0.2225806451612903

solver:  sag C:  1000.0


...

e:\university\dissertation\code\dissertations-2021-info\kasturi_mitra_find\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
e:\university\dissertation\code\dissertations-2021-info\kasturi_mitra_find\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
e:\university\dissertation\code\dissertations-2021-info\kasturi_mitra_find\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
e:\university\dissertation\code\dissertations-2021-info\kasturi_mitra_find\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did n

Confusion matrix: (True, False) 
 [[  57  439]
 [  49 1931]]
Accuracy:  0.802907915993538
Precision:  0.5377358490566038
Recall:  0.11491935483870967
F1 score:  0.18936877076411962

solver:  saga C:  100.0
Confusion matrix: (True, False) 
 [[  57  439]
 [  49 1931]]
Accuracy:  0.802907915993538
Precision:  0.5377358490566038
Recall:  0.11491935483870967
F1 score:  0.18936877076411962

solver:  saga C:  1000.0


e:\university\dissertation\code\dissertations-2021-info\kasturi_mitra_find\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
e:\university\dissertation\code\dissertations-2021-info\kasturi_mitra_find\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
e:\university\dissertation\code\dissertations-2021-info\kasturi_mitra_find\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
e:\university\dissertation\code\dissertations-2021-info\kasturi_mitra_find\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did n

Confusion matrix: (True, False) 
 [[  57  439]
 [  49 1931]]
Accuracy:  0.802907915993538
Precision:  0.5377358490566038
Recall:  0.11491935483870967
F1 score:  0.18936877076411962

solver:  saga C:  10000.0
Confusion matrix: (True, False) 
 [[  57  439]
 [  49 1931]]
Accuracy:  0.802907915993538
Precision:  0.5377358490566038
Recall:  0.11491935483870967
F1 score:  0.18936877076411962



e:\university\dissertation\code\dissertations-2021-info\kasturi_mitra_find\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


In [381]:
opt_model_name

('solver: ', 'liblinear', 'C: ', 10000.0)

In [305]:
X = data_original
y = X['startup']
X = X.drop(columns=['description_vector', 'country_code', 'startup'])
X = X.replace({'founding_year': {0: 2021}}) 
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)

model = make_pipeline(StandardScaler(), LogisticRegression(solver='liblinear', C=10000))
model.fit(X_train, y_train)
predicted = model.predict(X_test)
print("Confusion matrix: (True, False) \n", confusion_matrix(y_test, predicted, labels=[True, False]))
print("Accuracy: ", accuracy_score(y_test, predicted))
print("Precision: ", precision_score(y_test, predicted, pos_label=True))
print("Recall: ", recall_score(y_test, predicted, pos_label=True))
print("F1 score: ", f1_score(y_test, predicted, pos_label=True))
print("")

Confusion matrix: (True, False) 
 [[ 54  45]
 [ 21 376]]
Accuracy:  0.8669354838709677
Precision:  0.72
Recall:  0.5454545454545454
F1 score:  0.6206896551724138



### Use this seed to replicate results: 52049815

In [818]:
seed = 52049815 #np.random.randint(1000000000)
print("random_state = ", seed)

X = data_original
y = X['startup']
X = X.drop(columns=['description_vector', 'country_code', 'startup'])
X = X.replace({'founding_year': {0: 2021}}) 
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=seed)



model = make_pipeline(StandardScaler(), LogisticRegression(solver='liblinear', C=10000, random_state=seed))
model.fit(X_train, y_train)
predicted = model.predict(X_test)
print("Confusion matrix: (True, False) \n", confusion_matrix(y_test, predicted, labels=[True, False]))
print("Accuracy: ", accuracy_score(y_test, predicted))
print("Precision: ", precision_score(y_test, predicted, pos_label=True))
print("Recall: ", recall_score(y_test, predicted, pos_label=True))
print("F1 score: ", f1_score(y_test, predicted, pos_label=True))
print("")

random_state =  52049815
Confusion matrix: (True, False) 
 [[ 55  44]
 [ 24 373]]
Accuracy:  0.8629032258064516
Precision:  0.6962025316455697
Recall:  0.5555555555555556
F1 score:  0.6179775280898877



### Average stats over multiple seeds

In [825]:
X = data_original
y = X['startup']
X = X.drop(columns=['description_vector', 'country_code', 'startup'])
X = X.replace({'founding_year': {0: 2021}}) 
# X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)

accuracy, precision, recall, f1score = [], [], [], []
n = 10000

for i in range(n):
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)
    model = make_pipeline(StandardScaler(), LogisticRegression(solver='liblinear', C=10000))
    model.fit(X_train, y_train)
    predicted = model.predict(X_test)
    # print("Confusion matrix: (True, False) \n", confusion_matrix(y_test, predicted, labels=[True, False]))
    accuracy.append(accuracy_score(y_test, predicted))
    precision.append(precision_score(y_test, predicted, pos_label=True))
    recall.append(recall_score(y_test, predicted, pos_label=True))
    f1score.append(f1_score(y_test, predicted, pos_label=True))
print("Accuracy: ", sum(accuracy)/n)
print("Precision: ", sum(precision)/n)
print("Recall: ", sum(recall)/n)
print("F1 score: ", sum(f1score)/n)

Accuracy:  0.8401056451612964
Precision:  0.6496439370426383
Recall:  0.43785555555555394
F1 score:  0.5213727088461219


## SVM

#### Grid search through all kernels and penalty parameters (and degrees for poly)

In [384]:

X = data_original
y = X['startup']
X = X.drop(columns=['description_vector', 'country_code', 'startup'])
X = X.replace({'founding_year': {0: 2021}}) 
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)

kernels = ['rbf', 'poly', 'sigmoid', 'linear']
C = [1, 10, 100, 1000]
opt_model, opt_model_name, f1 = None, "empty", -1
for k in kernels:
    degrees = [1]
    C = [1, 10, 100, 1000]
    if k == "poly":
        degrees = [1, 2]
        C = [1, 10]
    if k == "linear":
        C = [1, 10]
    for d in degrees:
         for c in C:
            print ("kernel: ", k, " degree: ", d, " c: ", c)
            model = make_pipeline(StandardScaler(), SVC(C=c, kernel=k, degree=d))
#             model.fit(X_train, y_train)
#             predicted = model.predict(X_test)
            predicted = cross_val_predict(model, X, y, cv=5)
            print("Confusion matrix: (True, False) \n", confusion_matrix(y, predicted, labels=[True, False]))
            print("Accuracy: ", accuracy_score(y, predicted))
            print("Precision: ", precision_score(y, predicted, pos_label=True))
            print("Recall: ", recall_score(y, predicted, pos_label=True))
            print("F1 score: ", f1_score(y, predicted, pos_label=True))
            if f1_score(y, predicted, pos_label=True) > f1:
                f1 = f1_score(y, predicted, pos_label=True)
                opt_model = model
                opt_model_name = "kernel: ", k, " degree: ", d, " c: ", c
            print("")

kernel:  rbf  degree:  1  c:  1
Confusion matrix: (True, False) 
 [[ 111  385]
 [  69 1911]]
Accuracy:  0.8166397415185783
Precision:  0.6166666666666667
Recall:  0.22379032258064516
F1 score:  0.32840236686390534

kernel:  rbf  degree:  1  c:  10
Confusion matrix: (True, False) 
 [[ 209  287]
 [ 124 1856]]
Accuracy:  0.8340064620355412
Precision:  0.6276276276276276
Recall:  0.4213709677419355
F1 score:  0.5042219541616405

kernel:  rbf  degree:  1  c:  100
Confusion matrix: (True, False) 
 [[ 260  236]
 [ 171 1809]]
Accuracy:  0.8356219709208401
Precision:  0.6032482598607889
Recall:  0.5241935483870968
F1 score:  0.5609492988133764

kernel:  rbf  degree:  1  c:  1000
Confusion matrix: (True, False) 
 [[ 260  236]
 [ 160 1820]]
Accuracy:  0.840064620355412
Precision:  0.6190476190476191
Recall:  0.5241935483870968
F1 score:  0.5676855895196506

kernel:  poly  degree:  1  c:  1
Confusion matrix: (True, False) 
 [[   0  496]
 [   0 1980]]
Accuracy:  0.7996768982229402
Precision:  0.0
R

e:\university\dissertation\code\dissertations-2021-info\kasturi_mitra_find\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion matrix: (True, False) 
 [[   0  496]
 [   0 1980]]
Accuracy:  0.7996768982229402
Precision:  0.0
Recall:  0.0
F1 score:  0.0

kernel:  poly  degree:  2  c:  1


e:\university\dissertation\code\dissertations-2021-info\kasturi_mitra_find\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion matrix: (True, False) 
 [[   0  496]
 [   0 1980]]
Accuracy:  0.7996768982229402
Precision:  0.0
Recall:  0.0
F1 score:  0.0

kernel:  poly  degree:  2  c:  10


...

e:\university\dissertation\code\dissertations-2021-info\kasturi_mitra_find\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion matrix: (True, False) 
 [[ 307  189]
 [ 304 1676]]
Accuracy:  0.8008885298869144
Precision:  0.502454991816694
Recall:  0.6189516129032258
F1 score:  0.5546522131887985

kernel:  sigmoid  degree:  1  c:  10
Confusion matrix: (True, False) 
 [[ 247  249]
 [ 234 1746]]
Accuracy:  0.8049273021001615
Precision:  0.5135135135135135
Recall:  0.49798387096774194
F1 score:  0.5056294779938588

kernel:  sigmoid  degree:  1  c:  100
Confusion matrix: (True, False) 
 [[ 249  247]
 [ 245 1735]]
Accuracy:  0.8012924071082391
Precision:  0.5040485829959515
Recall:  0.5020161290322581
F1 score:  0.5030303030303032

kernel:  sigmoid  degree:  1  c:  1000
Confusion matrix: (True, False) 
 [[ 246  250]
 [ 239 1741]]
Accuracy:  0.8025040387722132
Precision:  0.5072164948453608
Recall:  0.4959677419354839
F1 score:  0.5015290519877674

kernel:  linear  degree:  1  c:  1
Confusion matrix: (True, False) 
 [[   0  496]
 [   0 1980]]
Accuracy:  0.7996768982229402
Precision:  0.0
Recall:  0.0
F1 scor

e:\university\dissertation\code\dissertations-2021-info\kasturi_mitra_find\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion matrix: (True, False) 
 [[   0  496]
 [   0 1980]]
Accuracy:  0.7996768982229402
Precision:  0.0
Recall:  0.0
F1 score:  0.0



e:\university\dissertation\code\dissertations-2021-info\kasturi_mitra_find\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [385]:
opt_model_name

('kernel: ', 'rbf', ' degree: ', 1, ' c: ', 1000)

In [405]:
# train using best model on 80% data
X = data_original
y = X['startup']
X = X.drop(columns=['description_vector', 'country_code', 'startup'])
X = X.replace({'founding_year': {0: 2021}}) 
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)

model = make_pipeline(StandardScaler(), SVC(C=1000, kernel='rbf'))
model.fit(X_train, y_train)
predicted = model.predict(X_test)
print("Confusion matrix: (True, False) \n", confusion_matrix(y_test, predicted, labels=[True, False]))
print("Accuracy: ", accuracy_score(y_test, predicted))
print("Precision: ", precision_score(y_test, predicted, pos_label=True))
print("Recall: ", recall_score(y_test, predicted, pos_label=True))
print("F1 score: ", f1_score(y_test, predicted, pos_label=True))
print("")

Confusion matrix: (True, False) 
 [[ 54  45]
 [ 18 379]]
Accuracy:  0.8729838709677419
Precision:  0.75
Recall:  0.5454545454545454
F1 score:  0.631578947368421



### Use this seed to replicate results = 228144966

In [817]:
seed = 228144966 #np.random.randint(1000000000)
print("random_state = ", seed)

X = data_original
y = X['startup']
X = X.drop(columns=['description_vector', 'country_code', 'startup'])
X = X.replace({'founding_year': {0: 2021}}) 
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=seed)

model = make_pipeline(StandardScaler(), SVC(C=1000, kernel='rbf', random_state=seed))
model.fit(X_train, y_train)
predicted = model.predict(X_test)
print("Confusion matrix: (True, False) \n", confusion_matrix(y_test, predicted, labels=[True, False]))
print("Accuracy: ", accuracy_score(y_test, predicted))
print("Precision: ", precision_score(y_test, predicted, pos_label=True))
print("Recall: ", recall_score(y_test, predicted, pos_label=True))
print("F1 score: ", f1_score(y_test, predicted, pos_label=True))
print("")

random_state =  228144966
Confusion matrix: (True, False) 
 [[ 57  42]
 [ 25 372]]
Accuracy:  0.8649193548387096
Precision:  0.6951219512195121
Recall:  0.5757575757575758
F1 score:  0.6298342541436464



### Average stats over multiple seeds

In [828]:
X = data_original
y = X['startup']
X = X.drop(columns=['description_vector', 'country_code', 'startup'])
X = X.replace({'founding_year': {0: 2021}}) 
# X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)

accuracy, precision, recall, f1score = [], [], [], []
n = 10000

for i in range(n):
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)
    model = make_pipeline(StandardScaler(), SVC(C=1000, kernel='rbf'))
    model.fit(X_train, y_train)
    predicted = model.predict(X_test)
    # print("Confusion matrix: (True, False) \n", confusion_matrix(y_test, predicted, labels=[True, False]))
    accuracy.append(accuracy_score(y_test, predicted))
    precision.append(precision_score(y_test, predicted, pos_label=True))
    recall.append(recall_score(y_test, predicted, pos_label=True))
    f1score.append(f1_score(y_test, predicted, pos_label=True))
print("Accuracy: ", sum(accuracy)/n)
print("Precision: ", sum(precision)/n)
print("Recall: ", sum(recall)/n)
print("F1 score: ", sum(f1score)/n)

Accuracy:  0.8409465725806454
Precision:  0.6233324421098922
Recall:  0.5184121212121264
F1 score:  0.5649229274158015


## Random Forest

In [225]:
X = data_original
y = X['startup']
X = X.drop(columns=['description_vector', 'country_code', 'startup'])
X = X.replace({'founding_year': {0: 2021}}) 

f1, opt_model_name = -1, ""

criteria = ['gini','entropy']
n_estimators = [10, 50, 100, 500, 1000]
for n_estimator in n_estimators:
    for criterion in criteria:
        for max_depth in [3, 5,  10, 20]:
            print("n_estimator: ", n_estimator, "Criteria: ", criterion, "Max depth: ", max_depth)
            model = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=n_estimator, criterion=criterion, max_depth=max_depth))
            #             model.fit(X_train, y_train)
            #             predicted = model.predict(X_test)
            predicted = cross_val_predict(model, X, y, cv=5)
            print("Confusion matrix: (True, False) \n", confusion_matrix(y, predicted, labels=[True, False]))
            print("Accuracy: ", accuracy_score(y, predicted))
            print("Precision: ", precision_score(y, predicted, pos_label=True))
            print("Recall: ", recall_score(y, predicted, pos_label=True))
            print("F1 score: ", f1_score(y, predicted, pos_label=True))
            if f1_score(y, predicted, pos_label=True) > f1:
                f1 = f1_score(y, predicted, pos_label=True)
                opt_model_name = "n_estimator: ", n_estimator, "Criteria: ", criterion, "Max depth: ", max_depth
            print("")

n_estimator:  10 Criteria:  gini Max depth:  3
Confusion matrix: (True, False) 
 [[ 270  226]
 [ 143 1837]]
Accuracy:  0.8509693053311793
Precision:  0.6537530266343826
Recall:  0.5443548387096774
F1 score:  0.594059405940594

n_estimator:  10 Criteria:  gini Max depth:  5
Confusion matrix: (True, False) 
 [[ 295  201]
 [ 159 1821]]
Accuracy:  0.8546042003231018
Precision:  0.6497797356828194
Recall:  0.594758064516129
F1 score:  0.6210526315789474

n_estimator:  10 Criteria:  gini Max depth:  10
Confusion matrix: (True, False) 
 [[ 266  230]
 [ 186 1794]]
Accuracy:  0.8319870759289176
Precision:  0.588495575221239
Recall:  0.5362903225806451
F1 score:  0.561181434599156

n_estimator:  10 Criteria:  gini Max depth:  20
Confusion matrix: (True, False) 
 [[ 234  262]
 [ 160 1820]]
Accuracy:  0.8295638126009693
Precision:  0.5939086294416244
Recall:  0.4717741935483871
F1 score:  0.5258426966292135

n_estimator:  10 Criteria:  entropy Max depth:  3
Confusion matrix: (True, False) 
 [[ 293

Confusion matrix: (True, False) 
 [[ 274  222]
 [ 148 1832]]
Accuracy:  0.8505654281098546
Precision:  0.6492890995260664
Recall:  0.5524193548387096
F1 score:  0.5969498910675382

n_estimator:  1000 Criteria:  entropy Max depth:  5
Confusion matrix: (True, False) 
 [[ 308  188]
 [ 172 1808]]
Accuracy:  0.8546042003231018
Precision:  0.6416666666666667
Recall:  0.6209677419354839
F1 score:  0.6311475409836066

n_estimator:  1000 Criteria:  entropy Max depth:  10
Confusion matrix: (True, False) 
 [[ 287  209]
 [ 183 1797]]
Accuracy:  0.8416801292407108
Precision:  0.6106382978723405
Recall:  0.5786290322580645
F1 score:  0.5942028985507247

n_estimator:  1000 Criteria:  entropy Max depth:  20
Confusion matrix: (True, False) 
 [[ 254  242]
 [ 178 1802]]
Accuracy:  0.8303715670436187
Precision:  0.5879629629629629
Recall:  0.5120967741935484
F1 score:  0.5474137931034483



In [227]:
opt_model_name

('n_estimator: ', 500, 'Criteria: ', 'entropy', 'Max depth: ', 5)

In [237]:
from sklearn.ensemble import RandomForestClassifier
X = data_original
y = X['startup']
X = X.drop(columns=['description_vector', 'country_code', 'startup'])
X = X.replace({'founding_year': {0: 2021}}) 
    
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)

model = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=5))
model.fit(X_train, y_train)
predicted = model.predict(X_test)
print("Confusion matrix: (True, False) \n", confusion_matrix(y_test, predicted, labels=[True, False]))
print("Accuracy: ", accuracy_score(y_test, predicted))
print("Precision: ", precision_score(y_test, predicted, pos_label=True))
print("Recall: ", recall_score(y_test, predicted, pos_label=True))
print("F1 score: ", f1_score(y_test, predicted, pos_label=True))
print("")

Confusion matrix: (True, False) 
 [[ 67  32]
 [ 28 369]]
Accuracy:  0.8790322580645161
Precision:  0.7052631578947368
Recall:  0.6767676767676768
F1 score:  0.690721649484536



### Use this seed to replicate results: 69711077 

In [816]:
seed = 69711077 #np.random.randint(1000000000)
print("random_state = ", seed)

X = data_original
y = X['startup']
X = X.drop(columns=['description_vector', 'country_code', 'startup'])
X = X.replace({'founding_year': {0: 2021}}) 
    
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=seed)

model = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=5, random_state=seed))
model.fit(X_train, y_train)
predicted = model.predict(X_test)
print("Confusion matrix: (True, False) \n", confusion_matrix(y_test, predicted, labels=[True, False]))
print("Accuracy: ", accuracy_score(y_test, predicted))
print("Precision: ", precision_score(y_test, predicted, pos_label=True))
print("Recall: ", recall_score(y_test, predicted, pos_label=True))
print("F1 score: ", f1_score(y_test, predicted, pos_label=True))
print("")

random_state =  69711077
Confusion matrix: (True, False) 
 [[ 62  37]
 [ 23 374]]
Accuracy:  0.8790322580645161
Precision:  0.7294117647058823
Recall:  0.6262626262626263
F1 score:  0.6739130434782609



### Average stats over multiple seeds

In [830]:
X = data_original
y = X['startup']
X = X.drop(columns=['description_vector', 'country_code', 'startup'])
X = X.replace({'founding_year': {0: 2021}}) 
# X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)

accuracy, precision, recall, f1score = [], [], [], []
n = 10000

for i in range(n):
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)
    model = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=5, random_state=seed))
    model.fit(X_train, y_train)
    predicted = model.predict(X_test)
    # print("Confusion matrix: (True, False) \n", confusion_matrix(y_test, predicted, labels=[True, False]))
    accuracy.append(accuracy_score(y_test, predicted))
    precision.append(precision_score(y_test, predicted, pos_label=True))
    recall.append(recall_score(y_test, predicted, pos_label=True))
    f1score.append(f1_score(y_test, predicted, pos_label=True))
print("Accuracy: ", sum(accuracy)/n)
print("Precision: ", sum(precision)/n)
print("Recall: ", sum(recall)/n)
print("F1 score: ", sum(f1score)/n)

Accuracy:  0.855421572580642
Precision:  0.6456740668982774
Recall:  0.6161363636363846
F1 score:  0.6294234582876894


# Final List of Startup Names

In [841]:
all_corepo_orgs_preprocessed = pd.read_csv('corepo_preprocessed.csv', index_col=0)
all_corepo_orgs = pd.read_csv('corepo_lookup_labelled.csv', index_col=0)

all_corepo_orgs_preprocessed = all_corepo_orgs_preprocessed.drop(columns=['description_vector', 'name', 'country_code' ])
all_corepo_orgs_preprocessed


,founding_year,funding_numeric,is_linkedin,employee_count
0,2019.0,4700000.0,1,2
1,1989.0,0.0,1,5
2,1974.0,0.0,1,5
3,1922.0,0.0,1,7
4,1998.0,0.0,1,5
...,...,...,...,...
9652,2006.0,481000000.0,1,4
9653,2003.0,55000000.0,1,4
9655,2005.0,840300000.0,1,4
9656,2007.0,20300000.0,1,2


In [847]:
model = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=500, criterion='entropy', max_depth=5, random_state=seed))
model.fit(X, y)

all_corepo_orgs['whether_startup'] = model.predict(all_corepo_orgs_preprocessed)

In [849]:
all_corepo_orgs.to_csv('unprocessed_corepo_predicted_labels.csv', encoding = 'utf-8-sig')